In [21]:
import numpy as np
from sklearn.metrics import mean_squared_error

float_formatter = lambda x: "%.5f" % x
np.set_printoptions(formatter={'float_kind':float_formatter})

def poly_kernel2(x,xp,alpha=1,degree=2):
    return np.power(x.dot(xp.T) + alpha, degree)

def linear_kernel2(x,xp):
    return x.dot(xp.T)

def gaussian_kernel2(x,xp,sigma=1):
    return np.exp(-(np.linalg.norm(x.T - xp.T, ord = 2) ** 2) / (2 * (sigma ** 2)))

# main regression routine
# will call kernel ridge with various params and find one with the lowest MSE
def regresskrr(inputval,outputval,targetset,truthset,ktype,lambdavals=[],sigmavals=[],alphavals=[],polyvals=[]):
    allresult = []
    for a in lambdavals:
        if ktype == 'linear':
            testpred = simplekrr(inputval,outputval,targetset,ktype='linear',lambdaval=a)
            mse = mean_squared_error(testpred,truthset)
            allresult.append([a, mse])
        elif ktype == 'poly':
            for b in alphavals:
                for c in polyvals:
                    testpred = simplekrr(inputval,outputval,targetset,ktype='poly',lambdaval=a, alphaval=b, poly_degree=c)
                    mse = mean_squared_error(testpred,truthset)
                    allresult.append([a, b, c, mse])
        elif ktype == 'gauss':
            for b in sigmavals:
                testpred = simplekrr(inputval,outputval,targetset,ktype='gauss',lambdaval=a,sigmaval=b)
                mse = mean_squared_error(testpred,truthset)
                allresult.append([a, b, mse])
    return np.array(allresult)

# kernel ridge routine
# will call linear/poly/gauss kernel as specified in the param
def simplekrr(inputval,outputval,targetset,ktype='poly',lambdaval=0,sigmaval=1,alphaval=1,poly_degree=2):
    
    # for poly: alpha, degree, lambda
    # for gauss: sigma, lambda
    # for linear: lambda

    # loop thru each target
    for n in targetset:
        #print ("calculate for target: ",n)
        # kxT section #
        # create x,xn arrays
        xval = np.copy(inputval).astype(float)
        #print(xval)

        # calculate kxT for them
        if ktype == 'poly':
            xvaltrans = poly_kernel2(xval,n,alphaval,poly_degree)
        elif ktype == 'linear':
            xvaltrans = linear_kernel2(xval,n)
        elif ktype == 'gauss':
            xvaltrans = np.array([])
            for idx,x in enumerate(xval):
                xvaltrans = np.append(xvaltrans,np.array([gaussian_kernel2(x,n,sigmaval)]))
        else:
            print('unknown kernel')
            XX
        #print('k_x_xprime:\n',xvaltrans)
        
        # (K+lambdaIN)^-1 section
        # calculate bigK

        for m in inputval:
            #print(m)
            SubK = np.copy(inputval).astype(float)
            if ktype == 'poly':
                SubKResult = poly_kernel2(SubK,m)
            elif ktype == 'linear':
                SubKResult = linear_kernel2(SubK,m)
            elif ktype == 'gauss':
                SubKResult = np.array([])
                for idx,x in enumerate(SubK):
                    SubKResult = np.append(SubKResult,np.array([gaussian_kernel2(x,m,sigmaval)]))
            else:
                print('unknown kernel')
                XX
            #print(SubKResult)
            if 'BigK' in locals():
                BigK = np.vstack([BigK,np.matrix(SubKResult)])
            else:
                BigK = np.matrix(SubKResult)
        #print('BigK ',BigK)
        # calculate lambdaIN
        identmat = np.identity(inputval.shape[0])
        lambdaIN = lambdaval*identmat
        #print(lambdaIN)
        if ktype == 'gauss':
            KlambdaINinv = np.linalg.inv(BigK)
        else:
            KlambdaINinv = np.linalg.inv(BigK + lambdaIN)
        #print('Klambdainv',KlambdaINinv)
        
        totval = xvaltrans * KlambdaINinv
        #print(totval)
        
        # t section
        tval = np.transpose(np.asmatrix(outputval))

        # final calculation
        result = xvaltrans * KlambdaINinv * tval
        
        del(BigK) # reset BigK for next iteration

        #print ("result is ",result.item(0))
        
        # append result to return value
        if 'retval' in locals():
            retval = np.append(retval,result.item(0))
        else:
            retval = np.array([result.item(0)])
    
    # Return the result(s)
    return retval

# very trivial dataset and testset/truth
mlinput = np.array([[2,3], [2,4], [3,5]]) # x_train
mloutput = np.array([6, 8, 15]) # y_train
mltest = np.array([[3,4],[4,5]]) # x_test
testtruth = np.array([12,20]) # the real/correct y value of x_test

# hyper parameters
lambdas = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1.0]
alphas = [1, 2, 3, 4, 5]
polys = [-1, 0, 0.5, 1, 2]
sigmas = [1e-5, 1e-4, 1e-3, 0.01, 0.1, 1, 3]

print('Y truth: ',testtruth)
print()

# polynomial kernel
result = regresskrr(mlinput,mloutput,mltest,testtruth,'poly',lambdas,sigmavals=[],alphavals=alphas,polyvals=polys)
result = result[result[:, 3].argsort()] # sort based on MSE
print('Best polynomial model parameter (lambda alpha poly MSE)',result[0,:])
best_poly_lambda = result[0,0] # grab the best parameters
best_poly_alpha = result[0,1]
best_poly_degree = result[0,2]
print('Y predict: ',simplekrr(mlinput,mloutput,mltest,ktype='poly',lambdaval=best_poly_lambda,
                              alphaval=best_poly_alpha,poly_degree=best_poly_degree))
print()

# linear kernel
result = regresskrr(mlinput,mloutput,mltest,testtruth,'linear',lambdas)
result = result[result[:, 1].argsort()] # sort based on MSE
print('Best linear model parameter (lambda MSE)',result[0,:])
best_linear_lambda = result[0,0] # grab the best parameters
print('Y predict: ',simplekrr(mlinput,mloutput,mltest,ktype='linear',lambdaval=best_linear_lambda))
print()

# gaussian kernel
result = regresskrr(mlinput,mloutput,mltest,testtruth,'gauss',lambdas,sigmavals=sigmas)
result = result[result[:, 2].argsort()] # sort based on MSE
print('Best gaussian model parameter (lambda sigma MSE)',result[0,:])
best_gauss_lambda = result[0,0]
best_gauss_sigma = result[0,1]
print('Y predict: ',simplekrr(mlinput,mloutput,mltest,ktype='gauss',lambdaval=best_gauss_lambda,
                              sigmaval=best_gauss_sigma))



Y truth:  [12 20]

Best polynomial model parameter (lambda alpha poly MSE) [1.00000 2.00000 2.00000 0.00009]
Y predict:  [11.98725 19.99571]

Best linear model parameter (lambda MSE) [0.00001 9.88334]
Y predict:  [11.88880 15.55542]

Best gaussian model parameter (lambda sigma MSE) [0.00100 3.00000 2.46667]
Y predict:  [12.79741 17.92696]
